In [1]:
import arcpy
from arcpy.management import *
from arcpy.conversion import *
from arcpy.da import *
from arcpy.sa import *
from arcpy import env
import os
import pandas as pd
import numpy as np

In [2]:
# 读取全数据点
point_data = r"D:\ArcgisData\pred_tl\pred_database\TL.gdb\SY_sample_pro"

In [3]:
[_.name for _ in arcpy.ListFields(point_data)]

['OBJECTID', 'Shape', '代码', 'Shape_Leng', 'A', 'B', 'C', 'D']

In [4]:
# 特征数据
fetatus_data = r"D:\ArcgisData\basedata\basetrain_30m.gdb"

In [5]:
# 设置工作环境
env.workspace = fetatus_data

In [6]:
# 采样
def sample_point(point_,raster_,out_name):
    Sample(raster_,point_,out_name,"NEAREST", "OBJECTID", "CURRENT_SLICE", None, '', None, None, "ROW_WISE", "TABLE")
    return None

# 导出CSV
def export_csv(table_,out_path,out_name):
    TableToTable(table_,out_path,out_name)
    return None
    

In [7]:
# 获取栅格列表
raster_list = arcpy.ListRasters()

In [10]:
table_gdb_path = r"D:\ArcgisData\pred_organic_p_n\feature_table\tableresult1.gdb"
# 使用Delete_management函数删除数据库中的所有内容
arcpy.Delete_management(table_gdb_path)
# 创建一个数据库
arcpy.management.CreateFileGDB(r"D:\ArcgisData\pred_organic_p_n\feature_table", "tableresult1", "CURRENT")

<Result 'D:\\ArcgisData\\pred_organic_p_n\\feature_table\\tableresult1.gdb'>

In [11]:
# 逐个采样
for one_raster in raster_list:
    sample_point(point_data,one_raster,os.path.join(table_gdb_path,one_raster))

In [12]:
env.workspace = table_gdb_path

In [13]:
result_df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(point_data,["OBJECTID","代码","A","B","C","D"]))
result_df.rename(columns={"OBJECTID":"SY_sample_pro"},inplace=True)
tables = arcpy.ListTables()
# 存储每个表的最后一个字段的数据
merged_data = []

# 读取每个表的最后一个字段的数据
for table in tables:
    
    # 将表转换为pandas数据帧
    df = pd.DataFrame(arcpy.da.TableToNumPyArray(table, "*"))
    # 提取最后一个字段的数据
    merged_df = df[['SY_sample_pro', df.columns[-1]]]
    # 合并
    result_df = pd.merge(result_df, merged_df, on=['SY_sample_pro'])


In [15]:
result_df

,SY_sample_pro,代码,A,B,C,D,DEM_Band_1,TWI_5_Band_1,TPI_201_Band_1,TPI_101_Band_1,...,CUR_Band_1,ASP_Band_1,PLCUR_Band_1,POCUR_Band_1,OSJL_Band_1,LAT_Band_1,LON_Band_1,DZ_Band_1,DL_Band_1,TRI_Band_1
0,1,6-1-4-2,6,1,4,2,912.900024,5.059266,-31.932800,-13.930603,...,3.199463,84.686363,0.848971,-2.350492,15.811388,2720980.0,454720.0,1,7,8.554887
1,3,7-1-3-2,7,1,3,2,927.650024,3.533883,13.445374,23.411438,...,1.600098,240.524078,-0.558773,-2.158871,122.983742,2730700.0,446800.0,2,3,25.156315
2,4,1-3-2-2,1,3,2,2,933.799988,3.996188,53.997803,30.658325,...,8.800049,335.675262,2.243167,-6.556881,176.776703,2736460.0,541660.0,2,3,5.391068
3,5,6-1-11-2,6,1,11,2,995.174988,5.168910,12.600708,-0.346008,...,1.200195,107.102776,-1.331749,-2.531944,79.056938,2737180.0,577660.0,2,2,17.364471
4,6,7-3-1-3,7,3,1,3,1248.875000,3.527490,81.788696,44.853760,...,6.800293,40.854885,3.371192,-3.429101,397.146057,2738620.0,478120.0,2,6,18.879101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,599,6-1-4-2,6,1,4,2,1510.724976,3.691726,41.067993,24.123657,...,-6.000000,241.798111,-2.472115,3.527885,232.647797,3138040.0,475240.0,6,6,24.666971
590,600,5-1-8-2,5,1,8,2,1469.449951,9.116919,-34.907593,-31.758667,...,-16.800781,282.042938,-0.036555,16.764227,210.950226,3138220.0,468940.0,6,6,21.665895
591,601,5-1-8-2,5,1,8,2,1492.250000,4.359059,24.126709,1.346558,...,2.000488,125.785538,1.033712,-0.966777,55.901699,3140020.0,485500.0,6,2,4.864552
592,602,6-1-4-2,6,1,4,2,1376.850098,4.029774,-18.826782,-4.640747,...,2.800293,123.231827,0.735143,-2.065150,225.000000,3145240.0,480460.0,2,3,13.933617


In [16]:

result_df.columns

Index(['SY_sample_pro', '代码', 'A', 'B', 'C', 'D', 'DEM_Band_1', 'TWI_5_Band_1',
       'TPI_201_Band_1', 'TPI_101_Band_1', 'TPI_11_Band_1', 'TPI_3_Band_1',
       'TMP_Band_1', 'SOILQS_Band_1', 'SLOP_Band_1', 'PRE_Band_1',
       'NIGTH_Band_1', 'NDVI_Band_1', 'CUR_Band_1', 'ASP_Band_1',
       'PLCUR_Band_1', 'POCUR_Band_1', 'OSJL_Band_1', 'LAT_Band_1',
       'LON_Band_1', 'DZ_Band_1', 'DL_Band_1', 'TRI_Band_1'],
      dtype='object')

In [17]:
new_columns = [str(_).replace("_Band_1","").replace("_","").replace("代码","TL") for _ in result_df.columns]

In [23]:
result_df.rename(columns=dict(zip(result_df.columns, new_columns)),inplace=True)
result_df

,SYsamplepro,TL,A,B,C,D,DEM,TWI5,TPI201,TPI101,...,CUR,ASP,PLCUR,POCUR,OSJL,LAT,LON,DZ,DL,TRI
0,1,6-1-4-2,6,1,4,2,912.900024,5.059266,-31.932800,-13.930603,...,3.199463,84.686363,0.848971,-2.350492,15.811388,2720980.0,454720.0,1,7,8.554887
1,3,7-1-3-2,7,1,3,2,927.650024,3.533883,13.445374,23.411438,...,1.600098,240.524078,-0.558773,-2.158871,122.983742,2730700.0,446800.0,2,3,25.156315
2,4,1-3-2-2,1,3,2,2,933.799988,3.996188,53.997803,30.658325,...,8.800049,335.675262,2.243167,-6.556881,176.776703,2736460.0,541660.0,2,3,5.391068
3,5,6-1-11-2,6,1,11,2,995.174988,5.168910,12.600708,-0.346008,...,1.200195,107.102776,-1.331749,-2.531944,79.056938,2737180.0,577660.0,2,2,17.364471
4,6,7-3-1-3,7,3,1,3,1248.875000,3.527490,81.788696,44.853760,...,6.800293,40.854885,3.371192,-3.429101,397.146057,2738620.0,478120.0,2,6,18.879101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,599,6-1-4-2,6,1,4,2,1510.724976,3.691726,41.067993,24.123657,...,-6.000000,241.798111,-2.472115,3.527885,232.647797,3138040.0,475240.0,6,6,24.666971
590,600,5-1-8-2,5,1,8,2,1469.449951,9.116919,-34.907593,-31.758667,...,-16.800781,282.042938,-0.036555,16.764227,210.950226,3138220.0,468940.0,6,6,21.665895
591,601,5-1-8-2,5,1,8,2,1492.250000,4.359059,24.126709,1.346558,...,2.000488,125.785538,1.033712,-0.966777,55.901699,3140020.0,485500.0,6,2,4.864552
592,602,6-1-4-2,6,1,4,2,1376.850098,4.029774,-18.826782,-4.640747,...,2.800293,123.231827,0.735143,-2.065150,225.000000,3145240.0,480460.0,2,3,13.933617


In [24]:
# 存储位置
feature_out_path = r"D:\ArcgisData\pred_organic_p_n\feature_table\feature_table_result"
result_df.to_csv(os.path.join(feature_out_path,"sy_tl_result.csv"),index=False)